# Multi-Class Conformal Prediction

### Getting Ext Repositories & Loading Libraries

In [1]:
!pip install --upgrade venn-abers

!pip install catboost
!pip install dtype_diet

#install OpenML
!pip install openml

  Preparing metadata (setup.py) ... done
  Created wheel for venn-abers: filename=venn_abers-1.4.6-py3-none-any.whl size=15121 sha256=b60aca0ab3767cbc11d4e88ef4a77a48aeac359509cf6866305b01b8bfa2c16d
  Stored in directory: /Users/joaquinromero/Library/Caches/pip/wheels/6e/11/f1/777527d5be7fc2085cb3c2deff203f6dccc1bb2c65c7d6cc8b
Successfully built venn-abers
  Using cached catboost-1.2.7-cp312-cp312-macosx_11_0_universal2.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached catboost-1.2.7-cp312-cp312-macosx_11_0_universal2.whl (27.0 MB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
  Using cached dtype_diet-0.0.2-py3-none-any.whl.metadata (6.1 kB)
Using cached dtype_diet-0.0.2-py3-none-any.whl (7.6 kB)
  Using cached openml-0.15.1-py3-none-any.whl.metadata (10 kB)
  Using cached liac_arff-2.5.0-py3-none-any.whl
  Using cached xmltodict-0.14.2-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached minio-7.2.15-py3-none-any.whl.metadata

In [4]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import plotly.io as pio
from plotly import tools
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# PyTorch
import torch

# Path & Fetching
import openml

# Date & Time
import time
from datetime import datetime

# Warnings
import warnings

# Notebook Optimizer
from tqdm import tqdm

# OS
from dtype_diet import report_on_dataframe, optimize_dtypes

# GC
import gc

#Scikit-Learn
from sklearn import svm

from sklearn.svm import SVC, LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.isotonic import IsotonicRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import CalibrationDisplay, calibration_curve
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score
from sklearn.metrics import average_precision_score, recall_score, f1_score, roc_auc_score, log_loss, brier_score_loss


# Categorical Boosting (CatBoost)
from catboost import CatBoostClassifier

# Light Gradient Boosting 
from lightgbm import LGBMClassifier

# VennAbers
from venn_abers import VennAbersCalibrator

# SciPy
from scipy.io import arff

In [5]:
# Google Colab Render
pio.renderers.default = 'colab'

# Seabor styling Set-Up
sns.set_style("whitegrid")

# Warnings
warnings.filterwarnings('ignore')

# Pandas Set-Up
pd.set_option('display.max_columns', 100)

In [6]:
init_notebook_mode(connected=True)

#### Checking CUDA GPU Support

In [7]:
is_cuda_available = torch.cuda.is_available()

print(f"CUDA (GPU support) is available: {is_cuda_available}")

CUDA (GPU support) is available: False


### Loading Dataset

In [8]:
datasets_df = openml.datasets.list_datasets(output_format="dataframe")
print(datasets_df.head(n=10))

datasets_df.set_index('did', inplace = True)

    did             name  version uploader  status format  MajorityClassSize  \
2     2           anneal        1        1  active   ARFF              684.0   
3     3         kr-vs-kp        1        1  active   ARFF             1669.0   
4     4            labor        1        1  active   ARFF               37.0   
5     5       arrhythmia        1        1  active   ARFF              245.0   
6     6           letter        1        1  active   ARFF              813.0   
7     7        audiology        1        1  active   ARFF               57.0   
8     8  liver-disorders        1        1  active   ARFF                NaN   
9     9            autos        1        1  active   ARFF               67.0   
10   10            lymph        1        1  active   ARFF               81.0   
11   11    balance-scale        1        1  active   ARFF              288.0   

    MaxNominalAttDistinctValues  MinorityClassSize  NumberOfClasses  \
2                           7.0                8